In [1]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
env = GridworldEnv()

In [3]:
env.nA

4

In [12]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a (prob, next_state, reward, done) tuple.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        old_v = V
        new_v = np.zeros(len(V))
        
        for state in range(env.nS):
            #weighted_action_values = []
            action_probabilities = []
            action_values = []
            
            for action in range(len(env.P[state])):
                transition_tuple = env.P[state][action][0]
                action_probability = policy[state][action]
                reward_action = transition_tuple[2]
                next_state = transition_tuple[1]
                action_values.append(reward_action +  discount_factor * old_v[next_state]) 
                # here only next state because there is only one successor state
                action_probabilities.append(action_probability)
            
            new_v[state] = np.sum(np.multiply(np.array(action_probabilities), np.array(action_values)))

        
        
        V = new_v
        if(np.max(np.abs(new_v - old_v)) <= theta):
            break
                
    return np.array(V)

In [13]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [14]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

In [15]:
print(v)

[  0.         -13.99989315 -19.99984167 -21.99982282 -13.99989315
 -17.99986052 -19.99984273 -19.99984167 -19.99984167 -19.99984273
 -17.99986052 -13.99989315 -21.99982282 -19.99984167 -13.99989315   0.        ]
